<img src="https://i.imgur.com/YxnDUwA.png" width=400>

Frequently we don't have access to any/enough data to perform accurate analysis, this is a common issue to a new/nich project. In those cases, we might need to find a way to collect data on our own.

__A Web Scraper__ is a program that extract data from a website.

### Problem Statement
- Build a Web Scraper to collect data about articles on [https://vnexpress.net/](https://vnexpress.net/).
- Required information:
  - Title
  - Description
  - Link to the Article
  - Link to Thumbnail Image (optional)

In [ ]:
# install selenium and other resources for crawling data
!pip install selenium
!apt-get update
# install other resources for doing crawling
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 25.8 MB/s 
     |████████████████████████████████| 139 kB 71.9 MB/s 
     |████████████████████████████████| 358 kB 49.5 MB/s 
     |████████████████████████████████| 55 kB 5.3 MB/s 
     |████████████████████████████████| 4.1 MB 40.8 MB/s 
     |████████████████████████████████| 58 kB 6.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.10 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
DRIVER = None

In [ ]:
def initialize_driver():
    global DRIVER
    if DRIVER is None:
        print('Initiating driver...')
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('-headless') 
        chrome_options.add_argument('-no-sandbox') 
        DRIVER = webdriver.Chrome('chromedriver',options=chrome_options)  # Create the new chrome browser with specific options
        print('Finished!')

In [ ]:
def close_driver():
    global DRIVER
    if DRIVER is not None:
        print('Quitting drive...')
        DRIVER.quit()
        print('Done')

    DRIVER = None

In [ ]:
close_driver()

In [ ]:
initialize_driver()

Initiating driver...
Finished!


In [ ]:
DRIVER.get('https://vnexpress.net/')
DRIVER.current_url

'https://vnexpress.net/'

# 🔬 ANALYZE

💡 **item-news**

![](https://i.imgur.com/sI6Slxi.png)


In [ ]:
all_news_elements = DRIVER.find_elements(By.CLASS_NAME, 'item-news')
len(all_news_elements)

184

To make it simple, we will do the **testing on 1 single article**. If this is sucessful, we will run the code on all the articles we found

In [ ]:
news_element = all_news_elements[9]

In [ ]:
print(news_element.get_attribute('outerHTML'))

<article class="item-news item-news-common off-thumb">
<a href="https://vnexpress.net/topic/cang-thang-nga-ukraine-25857" data-itm-source="#vn_source=Home&amp;vn_campaign=Banner-NgaTanCongUkraina&amp;vn_medium=Item-0&amp;vn_term=Desktop" title="Căng thẳng biên giới Ukraine - Nga" data-itm-added="1">
<img alt="Căng thẳng biên giới Ukraine - Nga" src="https://s1cdn.vnecdn.net/vnexpress/restruct/i/v626/v2_2019/pc/graphics/Nga-Ukraine-Pc.jpg">
</a>
</article>


We can see that all the information we need is inside `item-news` element
- Title
- Link to the article
- Description

Let's dive in

## Description

Within a description element, you can get the **article link**, **title** and **description**. Let's get them

In [ ]:
description_element = news_element.find_elements(By.CLASS_NAME, 'description')

StaleElementReferenceException: ignored

Get article link:

In [ ]:
a_element = news_element.find_element(By.TAG_NAME, 'a')
print(a_element)

<selenium.webdriver.remote.webelement.WebElement (session="6a317626ecb072f81cf2885e26a260bc", element="0a4d2548-94f3-4628-b463-9e8fda0669e8")>


In [ ]:
article_link = a_element.get_attribute('href')
print(article_link)

https://vnexpress.net/topic/cang-thang-nga-ukraine-25857


Get title:

In [ ]:
title = a_element.get_attribute('title')
print(title)

Căng thẳng biên giới Ukraine - Nga


Get description:

In [ ]:
a_element.text

''

Alright, we were able to get 3 out of 4 things that we want. At this point, you should **write a function** for reusability

In [ ]:
def get_link_title_description(news_element):
    '''
    Return link, title and description of an article from a web element
    '''

    description_element = news_element.find_element(By.CLASS_NAME, 'description')
    a_element           = description_element.find_element(By.TAG_NAME, 'a')

    # article link
    article_link = a_element.get_attribute('href')

    # title
    title = a_element.get_attribute('title')

    # description
    desc = a_element.text

    return article_link, title, desc

In [ ]:
news_element = all_news_elements[9]

a, b, c = get_link_title_description(news_element)

StaleElementReferenceException: ignored

In [ ]:
a, b, c

('https://vnexpress.net/gu-mac-thanh-lich-cua-truong-doan-bong-da-thai-lan-4463483.html',
 'Gu mặc thanh lịch của trưởng đoàn bóng đá Thái Lan',
 'Nualphan Lamsam, trưởng đoàn đội tuyển bóng đá Thái Lan, được các tạp chí thời trang nhận xét phong cách thanh lịch.')

## Thumb-art

Getting the thumbnail link is a little bit tricker, but it will be easier when we use the Inspection tool

In [ ]:
news_element = all_news_elements[8]

In [ ]:
thumbnail_link = news_element.find_element(By.TAG_NAME, 'img').get_attribute('src')
print(thumbnail_link)

https://vcdn1-kinhdoanh.vnecdn.net/2022/05/15/xangQuynhTran22022-1652547521-3876-1652547817.jpg?w=220&h=132&q=100&dpr=1&fit=crop&s=F2FIEYC0PKk1LIWQdi7_3w


In [ ]:
# thumbart_class = news_element.find_element(By.TAG_NAME, 'thumb-art')

Of course we will package everything we just wrote into a function as well

In [ ]:
def get_thumbnail_link(news_element):
    '''
    Return thumbnail link (if possible) given the web element
    '''
    
    thumbnail_link = ''
    try:
        thumbnail_link = news_element.find_element(By.TAG_NAME, 'img').get_attribute('src')
    except Exception:  # if there's an error
        print('Cannot find thumbnail_link')
    
    return thumbnail_link

In [ ]:
print(get_thumbnail_link(news_element))

https://vcdn1-kinhdoanh.vnecdn.net/2022/05/15/xangQuynhTran22022-1652547521-3876-1652547817.jpg?w=220&h=132&q=100&dpr=1&fit=crop&s=F2FIEYC0PKk1LIWQdi7_3w


# 🏃🏻‍♂️ LAB

## Requirements

You are now required to write a complete program to scrape **all** the articles at VNExpress. 

- An valid article must have **at least 3** information:
  1. Title
  2. Description
  3. Link to the article
  4. Link to thumbnail image (if possible)

- Your function should also count the number of valid articles it found.

- Please use the functions we define above. We created them for a reason.

In [ ]:
close_driver()
initialize_driver()

Quitting drive...
Done
Initiating driver...
Finished!


In [ ]:
DRIVER.get('https://vnexpress.net/')

In [ ]:
def scrape_vnexpress(DRIVER):

  all_news_elements = DRIVER.find_elements(By.CLASS_NAME, 'description')
  all_news_thumbnails = DRIVER.find_elements(By.CLASS_NAME, 'thumb-art')
  for i in all_news_elements:
    try:
      an_article = i.find_element(By.TAG_NAME, 'a')

      title = an_article.get_attribute('title')
      link = an_article.get_attribute('href')
      description = an_article.text

      print(title + "\n")
      print(link + "\n")
      print(description + "\n")
    finally:
      for x in all_news_thumbnails:        
        try:
          thumbnail_link = x.find_element(By.TAG_NAME, 'img').get_attribute('src')
          print(thumbnail_link)
        except Exception as e:
          print('Cannot find thumbnail_link')

          print("-"*30)


In [ ]:
  results = scrape_vnexpress(DRIVER)

NameError: ignored

In [ ]:
results[0:5]

TypeError: ignored

## Save results as csv file

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(results, columns=['link','title','description','thumbnail_link'])

In [ ]:
df.head()

In [ ]:
df.to_csv('vnexpress_scraped.csv', index=False)